### Random Acts of Pizza

In machine learning, it is often said there are no free lunches. How wrong we were.

This competition contains a dataset with 5671 textual requests for pizza from the Reddit community Random Acts of Pizza together with their outcome (successful/unsuccessful) and meta-data.

The task is to create an algorithm capable of predicting which requests will garner a cheesy (but sincere!) act of kindness.

In [2]:
#get data
import json
with open('train.json') as fin:
    trainjson = json.load(fin)

In [3]:
trainjson[0]

{'giver_username_if_known': 'N/A',
 'number_of_downvotes_of_request_at_retrieval': 0,
 'number_of_upvotes_of_request_at_retrieval': 1,
 'post_was_edited': False,
 'request_id': 't3_l25d7',
 'request_number_of_comments_at_retrieval': 0,
 'request_text': 'Hi I am in need of food for my 4 children we are a military family that has really hit hard times and we have exahusted all means of help just to be able to feed my family and make it through another night is all i ask i know our blessing is coming so whatever u can find in your heart to give is greatly appreciated',
 'request_text_edit_aware': 'Hi I am in need of food for my 4 children we are a military family that has really hit hard times and we have exahusted all means of help just to be able to feed my family and make it through another night is all i ask i know our blessing is coming so whatever u can find in your heart to give is greatly appreciated',
 'request_title': 'Request Colorado Springs Help Us Please',
 'requester_accoun

We are only interested in the text fields

### Input:
    - request_id: unique identifier for the request
    - request_title: title of the reddit post for pizza request
    - request_text-edit_aware: expository to request for pizza

### Output:
    - requester_received_pizza: whether requester gets his/her pizza

Currently, we are only using request_text as the input to build Naive Bayes classifier and the output is the requester_received_pizza field.

In [4]:
print('UID:\t', trainjson[0]['request_id'], '\n')
print('Title:\t', trainjson[0]['request_title'], '\n')
print('Text:\t', trainjson[0]['request_text_edit_aware'], '\n')
print('Tag:\t', trainjson[0]['requester_received_pizza'], end='\n')

UID:	 t3_l25d7 

Title:	 Request Colorado Springs Help Us Please 

Text:	 Hi I am in need of food for my 4 children we are a military family that has really hit hard times and we have exahusted all means of help just to be able to feed my family and make it through another night is all i ask i know our blessing is coming so whatever u can find in your heart to give is greatly appreciated 

Tag:	 False


### Converting json to pandas DataFrame

In [5]:
import pandas as pd
df = pd.io.json.json_normalize(trainjson)

df_train = df[['request_id', 'request_title','request_text_edit_aware','requester_received_pizza']]

df_train.head()

,request_id,request_title,request_text_edit_aware,requester_received_pizza
0,t3_l25d7,Request Colorado Springs Help Us Please,Hi I am in need of food for my 4 children we a...,False
1,t3_rcb83,"[Request] California, No cash and I could use ...",I spent the last money I had on gas today. Im ...,False
2,t3_lpu5j,"[Request] Hungry couple in Dundee, Scotland wo...",My girlfriend decided it would be a good idea ...,False
3,t3_mxvj3,"[Request] In Canada (Ontario), just got home f...","It's cold, I'n hungry, and to be completely ho...",False
4,t3_1i6486,[Request] Old friend coming to visit. Would LO...,hey guys:\n I love this sub. I think it's grea...,False


In [6]:
#getting the test data

import json

with open('test.json') as fin:
    testjson = json.load(fin)

In [7]:
print('UID:\t', testjson[0]['request_id'], '\n')
print('Title:\t', testjson[0]['request_title'], '\n')
print('Text:\t', testjson[0]['request_text_edit_aware'], '\n')
print('Tag:\t', testjson[0]['requester_received_pizza'], end='\n')

UID:	 t3_i8iy4 

Title:	 [request] pregger gf 95 degree house and no food.. promise to pay it forward! Northern Colorado 

Text:	 Hey all! It's about 95 degrees here and our kitchen is pretty much empty save for some bread and cereal.  My girlfriend/fiance is 8 1/2 months pregnant and we could use a good meal.  We promise to pay it forward when we get money! Thanks so much in advance! 



KeyError: 'requester_received_pizza'

In the test data, the label i.e. requester_received_pizza won't be known since that's the thing that our classifier is predicting. 

In [8]:
#converting test to DataFrame

df = pd.io.json.json_normalize(testjson)
df_test = df[['request_id','request_title','request_text_edit_aware']]
df_test.head()

,request_id,request_title,request_text_edit_aware
0,t3_i8iy4,[request] pregger gf 95 degree house and no fo...,Hey all! It's about 95 degrees here and our ki...
1,t3_1mfqi0,"[Request] Lost my job day after labour day, st...",I didn't know a place like this exists! \n\nI ...
2,t3_lclka,(Request) pizza for my kids please?,Hi Reddit. Im a single dad having a really rou...
3,t3_1jdgdj,[Request] Just moved to a new state(Waltham MA...,Hi I just moved to Waltham MA from my home sta...
4,t3_t2qt4,"[Request] Two girls in between paychecks, we'v...",We're just sitting here near indianapolis on o...


### Splitting training data before vectorization

The frist thing to do is to split our training data into two parts:
    - training: Use for trainig our model
    - validation: Use to check the "soundness" of the model built 

Splitting the data into 2 parts and holding out one part to check the model is one of method to validate the "soundness" of our model. It's call the hold-out validation.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

#train_test_split splits the data into 2 parts. We are spliting itto two set called traina nd valid split

train,valid = train_test_split(df_train, test_size = 0.2)


### Vectorize the train and validation set

In [26]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation

In [27]:
stopwords_en = stopwords.words('english')
stopwords_json = {"en":["a","a's","able","about","above","according","accordingly","across","actually","after","afterwards","again","against","ain't","all","allow","allows","almost","alone","along","already","also","although","always","am","among","amongst","an","and","another","any","anybody","anyhow","anyone","anything","anyway","anyways","anywhere","apart","appear","appreciate","appropriate","are","aren't","around","as","aside","ask","asking","associated","at","available","away","awfully","b","be","became","because","become","becomes","becoming","been","before","beforehand","behind","being","believe","below","beside","besides","best","better","between","beyond","both","brief","but","by","c","c'mon","c's","came","can","can't","cannot","cant","cause","causes","certain","certainly","changes","clearly","co","com","come","comes","concerning","consequently","consider","considering","contain","containing","contains","corresponding","could","couldn't","course","currently","d","definitely","described","despite","did","didn't","different","do","does","doesn't","doing","don't","done","down","downwards","during","e","each","edu","eg","eight","either","else","elsewhere","enough","entirely","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","f","far","few","fifth","first","five","followed","following","follows","for","former","formerly","forth","four","from","further","furthermore","g","get","gets","getting","given","gives","go","goes","going","gone","got","gotten","greetings","h","had","hadn't","happens","hardly","has","hasn't","have","haven't","having","he","he's","hello","help","hence","her","here","here's","hereafter","hereby","herein","hereupon","hers","herself","hi","him","himself","his","hither","hopefully","how","howbeit","however","i","i'd","i'll","i'm","i've","ie","if","ignored","immediate","in","inasmuch","inc","indeed","indicate","indicated","indicates","inner","insofar","instead","into","inward","is","isn't","it","it'd","it'll","it's","its","itself","j","just","k","keep","keeps","kept","know","known","knows","l","last","lately","later","latter","latterly","least","less","lest","let","let's","like","liked","likely","little","look","looking","looks","ltd","m","mainly","many","may","maybe","me","mean","meanwhile","merely","might","more","moreover","most","mostly","much","must","my","myself","n","name","namely","nd","near","nearly","necessary","need","needs","neither","never","nevertheless","new","next","nine","no","nobody","non","none","noone","nor","normally","not","nothing","novel","now","nowhere","o","obviously","of","off","often","oh","ok","okay","old","on","once","one","ones","only","onto","or","other","others","otherwise","ought","our","ours","ourselves","out","outside","over","overall","own","p","particular","particularly","per","perhaps","placed","please","plus","possible","presumably","probably","provides","q","que","quite","qv","r","rather","rd","re","really","reasonably","regarding","regardless","regards","relatively","respectively","right","s","said","same","saw","say","saying","says","second","secondly","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sensible","sent","serious","seriously","seven","several","shall","she","should","shouldn't","since","six","so","some","somebody","somehow","someone","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specified","specify","specifying","still","sub","such","sup","sure","t","t's","take","taken","tell","tends","th","than","thank","thanks","thanx","that","that's","thats","the","their","theirs","them","themselves","then","thence","there","there's","thereafter","thereby","therefore","therein","theres","thereupon","these","they","they'd","they'll","they're","they've","think","third","this","thorough","thoroughly","those","though","three","through","throughout","thru","thus","to","together","too","took","toward","towards","tried","tries","truly","try","trying","twice","two","u","un","under","unfortunately","unless","unlikely","until","unto","up","upon","us","use","used","useful","uses","using","usually","uucp","v","value","various","very","via","viz","vs","w","want","wants","was","wasn't","way","we","we'd","we'll","we're","we've","welcome","well","went","were","weren't","what","what's","whatever","when","whence","whenever","where","where's","whereafter","whereas","whereby","wherein","whereupon","wherever","whether","which","while","whither","who","who's","whoever","whole","whom","whose","why","will","willing","wish","with","within","without","won't","wonder","would","wouldn't","x","y","yes","yet","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","z","zero"]}
stopwords_json_en = set(stopwords_json['en'])
stopwords_nltk_en = set(stopwords.words('english'))
stopwords_punct = set(punctuation)
stoplist_combined = set.union(stopwords_json_en, stopwords_nltk_en, stopwords_punct)

In [28]:
wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 
    
def lemmatize_sent(text): 
    # Text input is string, returns lowercased strings.
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(word_tokenize(text))]

In [29]:
def preprocess_text(text):
    # Input: str, i.e. document/sentence
    # Output: list(str) , i.e. list of lemmas
    return [word for word in lemmatize_sent(text) 
            if word not in stoplist_combined
            and not word.isdigit()]


In [31]:
#Initializing the vectorizer and overriding the analyzer with the
#preprocess_text()


count_vect = CountVectorizer(analyzer = preprocess_text)

train_set = count_vect.fit_transform(train['request_text_edit_aware'])
train_tags = train['requester_received_pizza']

valid_set = count_vect.transform(valid['request_text_edit_aware'])
valid_tags = valid['requester_received_pizza']